In [1]:
!pip3 install ktrain

In [2]:
import ktrain
from ktrain import text
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from joblib import dump, load # for saving and loading sklearn object
from scipy.sparse import save_npz, load_npz # used for saving and loading sparse matrices
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
import os

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [31]:
df = pd.read_csv('IESAR_onehot.csv')
df.head()

,text,anger,disgust,fear,guilt,joy,sadness,shame
0,on day when feel close to my partner and other...,0,0,0,0,1,0,0
1,every time imagine that someone love or could ...,0,0,1,0,0,0,0
2,when had been obviously unjustly treated and h...,1,0,0,0,0,0,0
3,when think about the short time that we live a...,0,0,0,0,0,1,0
4,at gathering found myself involuntarily sittin...,0,1,0,0,0,0,0


In [0]:
def preprocess(X):
    documents = []
    stemmer = WordNetLemmatizer()

    for sen in range(0, len(X)):
        # Remove all the special characters
        document = re.sub(r'\W', ' ', str(X[sen]))

        # remove all single characters
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)

        # Remove single characters from the start
        document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 

        # Substituting multiple spaces with single space
        document = re.sub(r'\s+', ' ', document, flags=re.I)

        # Removing prefixed 'b'
        document = re.sub(r'^b\s+', '', document)

        # Converting to Lowercase
        document = document.lower()

        # Lemmatization
        document = document.split()

        document = [stemmer.lemmatize(word) for word in document]
        document = ' '.join(document)

        documents.append(document)
    return documents

df['text'] = preprocess(df['text'])

In [33]:
df.head()
# df.to_csv("ISEAR_onehot.csv", index = False)

,text,anger,disgust,fear,guilt,joy,sadness,shame
0,on day when feel close to my partner and other...,0,0,0,0,1,0,0
1,every time imagine that someone love or could ...,0,0,1,0,0,0,0
2,when had been obviously unjustly treated and h...,1,0,0,0,0,0,0
3,when think about the short time that we live a...,0,0,0,0,0,1,0
4,at gathering found myself involuntarily sittin...,0,1,0,0,0,0,0


In [69]:
(x_train, y_train), (x_test, y_test), preproc = text.texts_from_csv('IESAR_onehot.csv',
                      'text',
                      label_columns = ['anger', 'disgust', 'guilt', 'joy', 'sadness' ,'shame'],
                      val_filepath=None, # if None, 10% of data will be used for validation
                    #  max_features=NUM_WORDS, maxlen=MAXLEN,
                      ngram_range=1, maxlen = 63)

detected encoding: utf-8 (if wrong, set manually)
language: en
Word Counts: 7770
Nrows: 6701
6701 train sequences
train sequence lengths:
	mean : 20
	95percentile : 44
	99percentile : 64
x_train shape: (6701,63)
y_train shape: (6701, 6)
745 test sequences
test sequence lengths:
	mean : 19
	95percentile : 41
	99percentile : 55
x_test shape: (745,63)
y_test shape: (745, 6)


In [66]:
y_test

array([[1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0]])

In [62]:
model = text.text_classifier('fasttext', (x_train, y_train), preproc = preproc)
learner = ktrain.get_learner(model, train_data = (x_train, y_train), val_data = (x_test, y_test), batch_size = 100)

Is Multi-Label? False
compiling word ID features...
maxlen is 63
done.


In [65]:
# learner.fit_onecycle(2e-5, 1)
learner.autofit(0.0002, 17)



begin training using triangular learning rate policy with max lr of 0.0002...
Train on 6701 samples, validate on 745 samples
Epoch 1/17
6701/6701 [==============================] - 1s 185us/sample - loss: 1.1625 - accuracy: 0.4443 - val_loss: 1.0873 - val_accuracy: 0.5154
Epoch 2/17
6701/6701 [==============================] - 1s 189us/sample - loss: 1.1514 - accuracy: 0.4550 - val_loss: 1.0816 - val_accuracy: 0.5101
Epoch 3/17
6701/6701 [==============================] - 1s 185us/sample - loss: 1.1550 - accuracy: 0.4499 - val_loss: 1.0757 - val_accuracy: 0.5074
Epoch 4/17
6701/6701 [==============================] - 1s 191us/sample - loss: 1.1515 - accuracy: 0.4675 - val_loss: 1.0714 - val_accuracy: 0.4993
Epoch 5/17
6701/6701 [==============================] - 1s 189us/sample - loss: 1.1347 - accuracy: 0.4616 - val_loss: 1.0687 - val_accuracy: 0.4953
Epoch 6/17
6701/6701 [==============================] - 1s 185us/sample - loss: 1.1326 - accuracy: 0.4710 - val_loss: 1.0646 - val_ac

In [0]:
predictor = ktrain.get_predictor(learner.model, prepoc)

In [75]:
MODEL_NAME = 'bert-base-cased'
t = text.Transformer(MODEL_NAME, maxlen = 128, classes= ['anger', 'disgust', 'guilt', 'joy', 'sadness' ,'shame'], batch_size = 32)

trn = t.preprocess_train(x_train1.values, y_train.values)
val = t.preprocess_test(x_test1.values, y_test.values)
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data= trn, val_data = val, batch_size = 32)
learner.fit_onecycle(3e-5, 1)

/usr/local/lib/python3.6/dist-packages/ktrain/text/preprocessor.py:367: UserWarning: The class_names argument is replacing the classes argument. Please update your code.
  warnings.warn('The class_names argument is replacing the classes argument. Please update your code.')


preprocessing train...
language: en
train sequence lengths:
	mean : 20
	95percentile : 44
	99percentile : 63


/usr/local/lib/python3.6/dist-packages/ktrain/text/preprocessor.py:478: UserWarning: class_names argument was ignored, as they were extracted from string labels in dataset
  if self.get_classes(): warnings.warn('class_names argument was ignored, as they were extracted from string labels in dataset')


preprocessing test...
language: en
test sequence lengths:
	mean : 20
	95percentile : 42
	99percentile : 64




begin training using onecycle policy with max lr of 3e-05...
Train for 187 steps, validate for 47 steps
187/187 [==============================] - 8337s 45s/step - loss: 1.5445 - accuracy: 0.4058 - val_loss: 1.0465 - val_accuracy: 0.6436


In [0]:
# from sklearn.metrics import roc_auc_score, classification_report
# y_pred = learner.model.predict(x_test, verbose = 0)
# score = roc_auc_score(y_test, y_pred)
# class_report = classification_report(y_test, y_pred)

In [0]:
# predictor
predictor = ktrain.get_predictor(learner.model, t)

In [0]:
data = [ 'This movie was horrible! The plot was boring. Acting was okay, though.',
         'The film really sucked. I want my money back.',
        'The plot had too many holes.',
        'What a beautiful romantic comedy. 10/10 would see again!',
         ]

In [99]:
predictor.predict(data)

['sadness', 'sadness', 'shame', 'joy']

In [0]:
from sklearn.model_selection import train_test_split
x_train1, x_test1, y_train, y_test = train_test_split(df1['text'], df1['emotion'], test_size = 0.2)

In [87]:
learner.validate(class_names = t.get_classes())

              precision    recall  f1-score   support

       anger       0.51      0.55      0.53       201
     disgust       0.71      0.60      0.65       221
        fear       0.73      0.76      0.75       215
       guilt       0.49      0.59      0.54       223
         joy       0.74      0.93      0.82       209
     sadness       0.64      0.70      0.67       209
       shame       0.76      0.38      0.50       212

    accuracy                           0.64      1490
   macro avg       0.66      0.64      0.64      1490
weighted avg       0.66      0.64      0.64      1490



array([[111,  16,   7,  40,   7,  16,   4],
       [ 24, 132,  16,  21,  12,   7,   9],
       [ 11,   8, 164,   8,  12,  12,   0],
       [ 33,   4,  14, 131,   8,  27,   6],
       [  0,   2,   4,   2, 194,   5,   2],
       [  7,   5,   9,  18,  19, 147,   4],
       [ 31,  19,  11,  46,  11,  14,  80]])

In [0]:
predictor.save('bert_sentiment')

In [0]:
# Load the saved model
loaded_predictor = ktrain.load_predictor('bert_sentiment')

In [101]:
predictor.predict("I am feeling good and fantastic")

'joy'

In [106]:
loaded_predictor.predict("I am feeling good and fantastic")

'joy'

In [107]:
loaded_predictor.predict_proba("I am feeling good and fantastic")

array([0.01393803, 0.01612479, 0.02038617, 0.01535057, 0.89885956,
       0.01808071, 0.01726019], dtype=float32)

In [108]:
loaded_predictor.get_classes()

['anger', 'disgust', 'fear', 'guilt', 'joy', 'sadness', 'shame']